## A library to pull the paper content
Arxiv is a library to pull the content of papers from Arxiv.com

In [1]:
%pip install -qqq arxiv

Note: you may need to restart the kernel to use updated packages.


In [7]:
from IPython.display import Markdown

import arxiv

In [13]:
search = arxiv.Search(id_list=["1706.03762"])
paper = next(search.results())
print(paper.title)

Attention Is All You Need


In [14]:
Markdown(paper.summary)

The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 2 BLEU. On the WMT 2014 English-to-French
translation task, our model establishes a new single-model state-of-the-art
BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction
of the training costs of the best models from the literature. We show that the
Transformer generalizes well to other tasks by applying it successfully to
English constituency parsing both with large and limited training data.

In [15]:
%pip install -qqq langchain openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import getpass

openai_api_key = getpass.getpass("OpenAI API key: ")

In [17]:
import os

os.environ["OPENAI_API_KEY"] = openai_api_key

## text-davinci-003 vs gpt-3.5-turbo

text-davinci-003 and gpt-3.5-turbo are two different language models created by OpenAI. The main differences are:

text-davinci-003:

    It is a newer and larger model than gpt-3.5-turbo, trained on more data.
    It has 9.4 billion parameters, compared to 3.5 billion for gpt-3.5-turbo. More parameters usually means the model can capture more complex patterns.
    It achieves state-of-the-art results on many language tasks like summarization, question answering, word sense disambiguation, etc.
    It has a wider range of capabilities, covering both conversational and formal language.

gpt-3.5-turbo:

    It is an earlier and smaller model from OpenAI, launched in 2020.
    It has fewer parameters so may generate less coherent long-form text, but can still perform many language tasks reasonably well.
    It is optimized more for conversational language, so may be better at casual dialogue and humor.
    It tends to have a more casual and colloquial tone in its writing style.

In summary, text-davinci-003 is a more advanced, higher-capacity model that outperforms gpt-3.5-turbo on most language tasks. However, gpt-3.5-turbo can still be useful for some applications where a more informal or conversational style is needed.

In [18]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

In [19]:
from IPython.display import Markdown

In [20]:
Markdown(llm("hello, this is a test"))



Hello! It looks like you are testing something. What is the purpose of your test?

## Using langchain and OpenAI to get the summary of the paper by giving context

In [21]:
response = llm(f"""Here's a summary of a paper:
{paper.summary}

Based on that summary, what is attention?""")

Markdown(response)



Attention is a mechanism that connects the encoder and decoder in sequence transduction models. It helps the model to focus on specific parts of the input when making predictions and can improve the performance of the model.

## pypdf to load pdf content

In [22]:
%pip install -qqq pypdf

Note: you may need to restart the kernel to use updated packages.


In [23]:
paper_path = paper.download_pdf()

In [24]:
paper_path

'./1706.03762v5.Attention_Is_All_You_Need.pdf'

In [26]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(paper_path)
pages = loader.load_and_split()

In [28]:
len(pages)

16

In [30]:
content = "\n\n".join([page.page_content for page in pages[0:2]])

In [32]:
Markdown(content)

Attention Is All You Need
Ashish Vaswani
Google Brain
avaswani@google.comNoam Shazeer
Google Brain
noam@google.comNiki Parmar
Google Research
nikip@google.comJakob Uszkoreit
Google Research
usz@google.com
Llion Jones
Google Research
llion@google.comAidan N. Gomezy
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser
Google Brain
lukaszkaiser@google.com
Illia Polosukhinz
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
1 Introduction
Recurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks
in particular, have been ﬁrmly established as state of the art approaches in sequence modeling and
Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the ﬁrst Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and
efﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating
our research.
yWork performed while at Google Brain.
zWork performed while at Google Research.
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v5  [cs.CL]  6 Dec 2017

transduction problems such as language modeling and machine translation [ 35,2,5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
statesht, as a function of the previous hidden state ht 1and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
signiﬁcant improvements in computational efﬁciency through factorization tricks [ 21] and conditional
computation [ 32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [ 2,19]. In all but a few cases [ 27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for signiﬁcantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2 Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difﬁcult to learn dependencies between distant positions [ 12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].
3 Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,35].
Here, the encoder maps an input sequence of symbol representations (x1;:::;x n)to a sequence
of continuous representations z= (z1;:::;z n). Given z, the decoder then generates an output
sequence (y1;:::;y m)of symbols one element at a time. At each step the model is auto-regressive

## You can ask question on above pdf like below

In [63]:
response = llm(f"""Here's the first two pages of a paper:
{content}

Based on that content, what is multi-head attention?""")

Markdown(response)



Multi-head attention is a mechanism used in the Transformer model architecture which allows the model to draw global dependencies between input and output. It works by splitting the model's attention into multiple "heads" which each focus on different parts of the input and output sequence, thus allowing the model to process more information at once and improve its performance.

## Now we are using 
### FAISS
FAISS is a library for efficient similarity search and clustering of dense vectors, but it doesn't come preloaded with any data. You need to provide your own data in the form of vectors to perform similarity search using FAISS.

The data can be loaded into FAISS from various sources, such as local files, databases, or remote APIs, depending on the specific use case and the data storage requirements. Once the data is loaded into FAISS, it can be indexed and searched efficiently using the various indexing methods provided by the library.

FAISS is designed to work with large-scale datasets, so it can handle large amounts of data efficiently. It can also be used for real-time search by keeping the index in memory and updating it in real-time as new data becomes available.

Overall, FAISS relies on the data that is provided to it locally or from other sources, and it doesn't have any built-in mechanism to pull data from a real-time vector database for similarity search. However, it can be integrated with other tools and systems to access data from remote sources and perform real-time indexing and search.

### tiktoken
tiktoken is a fast BPE tokeniser for use with OpenAI's models.

Models don't see text like you and I, instead they see a sequence of numbers (known as tokens). Byte pair encoding (BPE) is a way of converting text into tokens. It has a couple desirable properties: It's reversible and lossless, so you can convert tokens back into the original text

In [34]:
%pip install -qqq faiss-cpu tiktoken

Note: you may need to restart the kernel to use updated packages.


In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [40]:
len(docs)

48

In [41]:
embeddings = OpenAIEmbeddings()

## Below are feeding the pdf content and the OpenAI embeddings to FAISS

In [43]:
db = FAISS.from_documents(docs, embeddings)

In [44]:
docs = db.similarity_search("What is attention?")

In [48]:
Markdown(docs[0].page_content)

Attention Visualizations
Input-Input Layer5
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
Figure 3: An example of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of
the verb ‘making’, completing the phrase ‘making...more difﬁcult’. Attentions here shown only for
the word ‘making’. Different colors represent different heads. Best viewed in color.
13

## using load_qa_with_sources_chain

The load_qa_with_sources_chain() method in the langchain library is used to load question-answer pairs along with their source documents. It retrieves questions, answers and source documents from a data source and prepares them for processing by the langchain library.

In [49]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [50]:
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
query = "What is attention?"
sources = db.similarity_search(query)

In [56]:
sources[0]

Document(page_content='Attention Visualizations\nInput-Input Layer5\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nIt\nis\nin\nthis\nspirit\nthat\na\nmajority\nof\nAmerican\ngovernments\nhave\npassed\nnew\nlaws\nsince\n2009\nmaking\nthe\nregistration\nor\nvoting\nprocess\nmore\ndifficult\n.\n<EOS>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\n<pad>\nFigure 3: An example of the attention mechanism following long-distance dependencies in the\nencoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of\nthe verb ‘making’, completing the phrase ‘making...more difﬁcult’. Attentions here shown only for\nthe word ‘making’. Different colors represent different heads. Best viewed in color.\n13', metadata={'source': './1706.03762v5.Attention_Is_All_You_Need.pdf', 'page': 12})

In [58]:
results = chain({"input_documents": sources, "question": query}, return_only_outputs=True)

In [61]:
Markdown(results['output_text'])

 Attention is an integral part of sequence modeling and transduction models that allow for modeling of dependencies without regard to their distance in the input or output sequences.
SOURCES: 1706.03762v5.Attention_Is_All_You_Need.pdf

### Above is showing the source, from where did it get the answer for the given question. 